In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from progressbar import ProgressBar
import random
import matplotlib.pyplot as plt

import os
import tensorflow as tf
import tensorflow.compat.v1 as tfv1
from Integrated_Gradients_algorithm import *
from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
# config=tf.ConfigProto()
# config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
# sess=tf.Session(config=config)
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config)

In [2]:
import keras
def get_loss_gradients(img_input, model, target_one_hot, from_logits=False):
    images = tf.cast(img_input, tf.float32)
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=from_logits)
    with tf.GradientTape() as tape:
        tape.watch(images)
        preds = model(images)
        loss = cce(target_one_hot, preds)
#         top_class = preds[:, top_pred_idx]

    grads = tape.gradient(loss, images)
    return grads

In [3]:
def get_acc(predict_label, ground_truth=None):
    for i in predict_label[0]:
        if i[0] == ground_truth:
            return True
    return False

def softmax( f ):
    # instead: first shift the values of f so that the highest number is 0:
    f -= np.max(f) # f becomes [-666, -333, 0]
    return np.exp(f) / np.sum(np.exp(f))  # safe to do, gives the correct answer

def get_APFD(Gini_indexs, ground_truth_label, predicted_confidence, top_set=None):
    o_i = 0
    pbar = ProgressBar()
    wrong_num = 0
    for i in pbar(range(0, len(Gini_indexs))):
        if top_set is not None:
            if not get_acc(predict_label=decode_predictions(predicted_confidence[Gini_indexs[i]], top=top_set), 
                           ground_truth=ground_truth_label[Gini_indexs[i]]):
                o_i = o_i+i
#                 print(i, o_i)
                wrong_num = wrong_num+1
        else:
            if np.argmax(ground_truth_label[Gini_indexs[i]]) != np.argmax(predicted_confidence[Gini_indexs[i]]):
                o_i = o_i+i
                wrong_num = wrong_num+1
    APFD = 1 - o_i/(len(Gini_indexs)*wrong_num) + 1/(2*len(Gini_indexs))
    return APFD, len(Gini_indexs), wrong_num

def get_RAUC(Gini_indexs, ground_truth_label, predicted_confidence, top_set=None):
    pre_y_axis = []
    o_i = 0
    wrong_num = 0
    pbar = ProgressBar()
    for i in pbar(range(0, len(Gini_indexs))):
        if top_set is not None:
            if not get_acc(predict_label=decode_predictions(predicted_confidence[Gini_indexs[i]], top=top_set), 
                           ground_truth=ground_truth_label[Gini_indexs[i]]):  
                o_i = o_i+1
                wrong_num = wrong_num+1
                pre_y_axis.append(o_i)
            else:
                pre_y_axis.append(o_i)
        else:
            if np.argmax(ground_truth_label[Gini_indexs[i]]) != np.argmax(predicted_confidence[Gini_indexs[i]]):
                o_i = o_i+1
                wrong_num = wrong_num+1
                pre_y_axis.append(o_i)
            else:
                pre_y_axis.append(o_i)
    true_y_axis = wrong_num*(len(Gini_indexs)-wrong_num) + (wrong_num+1)*wrong_num/2
    RAUC = np.sum(pre_y_axis)/true_y_axis
#     print("RAUC: ", RAUC)
    return RAUC, len(Gini_indexs), wrong_num


In [4]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train = np.reshape(X_train,[-1,28*28])
X_test = np.reshape(X_test,[-1,28*28])
Y_train = to_categorical(Y_train,10)
Y_test = to_categorical(Y_test,10)

In [5]:
MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
model = load_model(MODEL_PATH+"fmnist_FC5_good.h5")

In [6]:
loss_acc = model.evaluate(X_test, Y_test, batch_size=128)
print('accuracy', loss_acc[-1])

79/79 [==============================] - 2s 2ms/step - loss: 64.3066 - accuracy: 0.8529
accuracy 0.8529000282287598


In [7]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_30 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_32 (Dense)             (None, 64)                65600     
_________________________________________________________________
dense_33 (Dense)             (None, 10)                650       
Total params: 1,608,922
Trainable params: 1,608,922
Non-trainable params: 0
_________________________________________________________________


In [7]:





max_iter = 1e+2
class_num = 10
decay_factor_gradent = 0.1
decay_factor_epsilon_lr = 0.99
# input_tensors = BE.placeholder(shape=(None, 28*28))
# grads = BE.gradients(loss = model.total_loss, variables = model.inputs)
x_GE_all = []

for label_i in range(0,class_num):
    x_GE = np.zeros((1, 28*28))
#     per_tmp = model.predict(x_GE)
#     label_tmp = np.argmax(per_tmp)
#     print('label:', label_tmp, '\t label confidence:', per_tmp[0,label_tmp])
    grads = get_gradients(img_input=x_GE, top_pred_idx=label_i)
#     grads = BE.gradients(loss = model.layers[-1].output[:,label_i], variables = model.inputs)
#     get_gradients = BE.function(inputs=model.inputs[0], outputs=grads)
#     x_moment = 0
#     epsilon_lr = 5e-1
#     for iter_i in range(0, int(max_iter)):
#         x_grad = get_gradients(x_GE)[0]
#         x_moment = x_moment * decay_factor_gradent + x_grad
#         epsilon_lr = epsilon_lr * decay_factor_epsilon_lr
#         x_GE = x_GE + x_moment * epsilon_lr
#         x_GE = np.clip(x_GE, 0, 1)
#         per_tmp = model.predict(x_GE)
#         if 1-per_tmp[0, label_i]<1e-6 :
#             break
#     label_tmp = np.argmax(per_tmp)
#     print('label:', label_tmp, '\t label confidence:', per_tmp[0,label_tmp], '\t iterations:',iter_i)
#     x_GE_all.append(x_GE)

TypeError: get_gradients() missing 1 required positional argument: 'model'

In [24]:
# grads = BE.gradients(loss = model.layers[-1].output[:,:], variables = model.inputs)

In [47]:
img_num = 10000
X_test1 = X_test[0:img_num]
Y_test1 = Y_test[0:img_num]
Gini = []
predicted_confidence = []
pbar = ProgressBar()
for x_tmp in pbar(X_test1):
    per_tmp = model.predict(x_tmp.reshape([-1,28*28]))
    Gini_tmp = 1-np.sum(per_tmp*per_tmp)
#     Gini_tmp = -np.sum(per_tmp*np.log2(per_tmp))
    Gini.append(Gini_tmp)
    predicted_confidence.append(per_tmp)
    
indexs = np.argsort(Gini)
indexs = indexs[::-1]

APFD,_,wrong_number = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('APFD:', APFD)
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('RAUC:', RAUC)

100% |########################################################################|
100% |########################################################################|


APFD: 0.5119218558803534


RAUC: 0.5524790673290377


In [34]:
print('test number:', len(Gini), '\t misclassified number:', wrong_number)

test number: 500 	 misclassified number: 69


(1000, 784)

In [48]:
# use one layer's activation
# get_activations = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])
Gini_act = []
grads_pre1 = []
grads_pre2 = []
grads_CE = []
grads_CE2 = []
num_class = 10
# X_test1 = X_test[0:10]
# Y_test1 = Y_test[0:10]
pbar = ProgressBar()
for x_tmp in pbar(X_test1):
    x_tmp = x_tmp.reshape([-1,28*28])
#     x_act = []
#     for label_i in range(num_class):
#         grads = get_gradients(img_input=x_tmp.reshape([-1,28*28]), model=model, top_pred_idx=label_i)
#         x_act.append(np.sum(np.abs(grads)))
#     x_act = np.array(x_act).reshape(-1, num_class)
#     x_act = softmax(x_act)
#     Gini_tmp = 1-np.sum(x_act*x_act)   
#     Gini_act.append(Gini_tmp)
    preds = model.predict(x_tmp)
    
    #     使用预测标签当做真实标签验证
    label1 = np.argmax(preds[0])
    grads = get_gradients(img_input=x_tmp, model=model, top_pred_idx=label1)
    grads = np.sum(np.abs(grads))
    grads_pre1.append(grads)
    
    target_one_hot = np.reshape(to_categorical(label1, num_class), (-1, num_class))
    grads = get_loss_gradients(img_input=x_tmp, model=model, target_one_hot=target_one_hot)
    grads = np.sum(np.abs(grads))
    grads_CE.append(grads)
    
    preds[0][label1] = 0
    label2 = np.argmax(preds[0])
    grads = get_gradients(img_input=x_tmp, model=model, top_pred_idx=label2)
    grads = np.sum(np.abs(grads))
    grads_pre2.append(grads)
    
    target_one_hot = np.reshape(to_categorical(label2, num_class), (-1, num_class))
    grads = get_loss_gradients(img_input=x_tmp, model=model, target_one_hot=target_one_hot)
    grads = np.sum(np.abs(grads))
    grads_CE2.append(grads)
    
#     preds[0][label1] = 0
#     label2 = np.argmax(preds[0])
#     grads = get_gradients(img_input=x_tmp, model=model, top_pred_idx=label2)
#     grads_pre2.append(grads)
    
# #     grads = get_integrated_gradients(img_input=x_tmp, model=model, top_pred_idx=tf.argmax(preds[0]),num_steps=10, img_size=(1,28*28))
#     grads = get_gradients(img_input=x_tmp, model=model, top_pred_idx=tf.argmax(preds[0]))
# #     grads_min = get_integrated_gradients(img_input=x_tmp, model=model, top_pred_idx=tf.argmin(preds[0]),num_steps=10)
# #     x_act = np.sum(np.abs(grads)) * np.sum(np.abs(grads_min))
#     x_act = np.sum(np.abs(grads))

#     grads = get_gradients(img_input=x_tmp.reshape([-1,28*28]), model=model, top_pred_idx=label_i)
#     x_act.append(np.sum(np.abs(grads)))
#     x_act = np.array(x_act).reshape(-1, num_class)
#     x_act = softmax(x_act)
#     Gini_tmp = 1-np.sum(x_act*x_act)   
#     Gini_act.append(x_act)
        
#     # 使用attention机制
#     x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
#     att1 = np.dot(np.transpose(x_act), x_act)
#     att_weight = np.sum(att1, axis = 1).reshape(np.shape(x_act))
#     x_act_ = softmax(x_act*att_weight)
#     Gini_tmp = 1-np.sum(x_act_*x_act_)
#     Gini_tmp = -np.sum(x_act_*np.log2(x_act_))
#     Gini_act.append(Gini_tmp)
    
#     # 未使用attention机制
#     x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
# #     Gini_tmp = 1-np.sum(x_act*x_act)
#     Gini_tmp = -np.sum(x_act*np.log2(x_act))
#     Gini_act.append(Gini_tmp)

# # use all layers' activation
# pbar = ProgressBar()
# get_activations1 = BE.function(inputs=model.inputs[0], outputs=model.layers[-1].output[:,:])
# get_activations2 = BE.function(inputs=model.inputs[0], outputs=model.layers[-2].output[:,:])
# get_activations3 = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])
# Gini_act = []
# for x_tmp in pbar(X_test):
#     x_act1 = softmax(get_activations1(x_tmp.reshape([-1,28*28])))
#     x_act2 = softmax(get_activations2(x_tmp.reshape([-1,28*28])))
#     x_act3 = softmax(get_activations3(x_tmp.reshape([-1,28*28])))
# #     Gini_tmp = 1-np.sum(x_act3*x_act3)
# #     Gini_tmp = 1-(np.sum(x_act1*x_act1)+np.sum(x_act2*x_act2)+np.sum(x_act3*x_act3))/3
#     Gini_tmp = -np.sum(x_act3*np.log2(x_act3))
# #     Gini_tmp = (-np.sum(x_act1*np.log2(x_act1))-np.sum(x_act2*np.log2(x_act2))-np.sum(x_act3*np.log2(x_act3)))/3
#     Gini_act.append(Gini_tmp)
    

100% |########################################################################|


In [49]:
Gini_act =[]
pbar = ProgressBar()
for i in pbar(range(len(grads_CE))):
#     x_act = np.sum(np.abs(grads_CE[i]))
    x_act =[np.sum(np.abs(grads_CE[i])),
            np.sum(np.abs(grads_CE2[i])),
#             np.sum(np.abs(grads_CE_min[i])),
#             np.sum(np.abs(grads_RCE[i])),
#             np.sum(np.abs(grads_CE_ave[i])),
            np.sum(np.abs(grads_pre1[i])),
            np.sum(np.abs(grads_pre2[i])),
#             np.sum(np.abs(grads_pre_min[i]))
           ]
#     print(i,':',x_act)
    Gini_act.append(x_act)
    
Gini_act = np.array(Gini_act)
indexs = np.argsort(np.abs(Gini_act[:,0]/(np.abs(Gini_act[:,1]))))
indexs = indexs[::-1]
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_test1, 
                predicted_confidence=np.array(predicted_confidence))
print('APFD:', APFD)
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_test1, 
                predicted_confidence=np.array(predicted_confidence))
print('RAUC:', RAUC)

100% |########################################################################|
100% |########################################################################|


APFD: 0.5743930319510536
RAUC: 0.619906132704861


In [30]:
tf.argmax(preds[0])

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [32]:
Y_test[199]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
get_integrated_gradients?